In [1]:
import glob
import os
import tokenizers
from tokenizers import Tokenizer

In [5]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !chmod 600 ~/.kaggle/kaggle.json
# !mkdir corpus
# !kaggle datasets download -d disisbig/hindi-wikipedia-articles-172k
# !unzip /content/hindi-wikipedia-articles-172k.zip -d /content/corpus

cp: kaggle.json: No such file or directory
chmod: /Users/kvzm411/.kaggle/kaggle.json: No such file or directory
mkdir: corpus: File exists
zsh:1: command not found: kaggle
unzip:  cannot find or open /content/hindi-wikipedia-articles-172k.zip, /content/hindi-wikipedia-articles-172k.zip.zip or /content/hindi-wikipedia-articles-172k.zip.ZIP.


In [3]:
# Get list of training and validation text files
train_files = glob.glob("/content/corpus/train/train/*.txt")
valid_files = glob.glob("/content/corpus/valid/valid/*.txt")

def concatenate_files(file_list, output_path):
    """
    Concatenates text files into a single file with two newlines between each file's content.

    Args:
    - file_list (list of str): List of file paths to be concatenated.
    - output_path (str): Path to the output file.
    """
    try:
        with open(output_path, "wb") as outfile:
            for file_path in file_list:
                try:
                    with open(file_path, "rb") as infile:
                        outfile.write(infile.read())
                        outfile.write(b"\n\n")
                except FileNotFoundError:
                    print(f"File not found: {file_path}")
                except Exception as e:
                    print(f"Error reading {file_path}: {e}")
        print(f"Successfully concatenated files into {output_path}")
    except Exception as e:
        print(f"Error writing to {output_path}: {e}")

# Concatenate train and validation files
concatenate_files(train_files, "/content/corpus/train/full.txt")
concatenate_files(valid_files, "/content/corpus/valid/full_val.txt")

Error writing to /content/corpus/train/full.txt: [Errno 2] No such file or directory: '/content/corpus/train/full.txt'
Error writing to /content/corpus/valid/full_val.txt: [Errno 2] No such file or directory: '/content/corpus/valid/full_val.txt'


In [4]:
corpus_file = "/content/corpus/train/full.txt"
tokenizer = tokenizers.SentencePieceBPETokenizer()
tokenizer.train(corpus_file)
tokenizer.save("hindi_bpe_tokenizer.json")

Exception: No such file or directory (os error 2)

In [ ]:
tokenizer = Tokenizer.from_file("hindi_bpe_tokenizer.json")

def compression_ratio(text, tokenizer):
    encoded = tokenizer.encode(text)
    compressed_length = len(encoded.ids)
    original_length = len(text)
    return original_length / compressed_length

sample_text = "यह एक उदाहरण पाठ है। हम बीपीई टोकनाइज़र का उपयोग करेंगे।"
encoded_text = tokenizer.encode(sample_text)
compression = compression_ratio(sample_text, tokenizer)

print(f"Encoded text: {encoded_text.tokens}")
print(f"Vocabulary size: {len(tokenizer.get_vocab())}")
print(f"Compression ratio: {compression}")
print(f"Decoded text: {tokenizer.decode(encoded_text.ids)}")

Encoded text: ['▁यह', '▁एक', '▁उदाहरण', '▁पाठ', '▁है।', '▁हम', '▁बी', 'पी', 'ई', '▁टोक', 'नाइ', 'ज़र', '▁का', '▁उपयोग', '▁करेंगे।']
Vocabulary size: 30000
Compression ratio: 3.7333333333333334
Decoded text: यह एक उदाहरण पाठ है। हम बीपीई टोकनाइज़र का उपयोग करेंगे।
